In [ ]:
import os
import boto3
import pandas as pd
from src.dataloader import DataLoader
from src.feature_generator import FeatureEngineering
from src.config import PATHS, QUERY, KEYS, MLCONFIG
from src.helpers import (check_s3_bucket, generate_result_from_query,
                         setup_default_clients)

In [ ]:
AWS_DEFAULT_REGION = os.environ["AWS_DEFAULT_REGION"]
AWS_ACCESS_KEY_ID = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]

In [ ]:
s3, athena = setup_default_clients()

In [ ]:
s3_resource = boto3.resource(
    "s3",
    region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

check_s3_bucket(s3_resource, PATHS.BUCKET)

In [ ]:
# list folders / objects in s3 bucket
response = s3.list_objects(Bucket=PATHS.BUCKET)
[content["Key"] for content in response["Contents"]]

In [ ]:
# Generate query
# generate_response(
#     client=athena,
#     query=QUERY.RAW_DATA,
#     output="s3://smu-is614-iot-step-tracker/queries/",
# )

In [ ]:
session = boto3.setup_default_session(
        region_name=KEYS.AWS_DEFAULT_REGION,
        aws_access_key_id=KEYS.AWS_ACCESS_KEY_ID,
        aws_secret_access_key=KEYS.AWS_SECRET_ACCESS_KEY,
    )

In [ ]:
QUERY = """
    SELECT
        *,
        CASE 
          WHEN uuid LIKE '%_walk_%' THEN true 
          ELSE false 
        END AS target
    FROM
        "smu-iot"."microbit"
    WHERE
        seconds IS NOT null
    ORDER BY
        uuid, timestamp, seconds    
"""

In [ ]:
dataloader = DataLoader(session=session)
dataloader.load_data(query=QUERY, database=PATHS.DB_TEST)
print(dataloader.get_data())

In [ ]:
df = dataloader.data.copy()
df.to_csv(PATHS.DATA_DIR / "test.csv", index=False)

In [ ]:
###
df = pd.read_csv(PATHS.DATA_DIR / "test.csv")

In [ ]:
df.head()

In [ ]:
df_copy = df.copy()

In [ ]:
fe_settings = {
    "upload_to_s3": False, 
    "apply_smooth_filter": False,
    "apply_median_filter": False, 
    "apply_savgol_filter": False,
    "extract_features": False
}
fe = FeatureEngineering(**fe_settings)